# Chain

In [1]:
import sys
sys.path.append('../')
from botcore.setup import get_ai21_model, enable_tracing
from botcore.utils.json_parser import parse_nested_json
from botcore.utils.memory_utils import QAMemory
from botcore.utils.prompt_utils import build_prompt
MODEL = get_ai21_model(max_tokens = 400)
enable_tracing("assess")


SAMPLE = '\n\n```\n[\n  {\n    "question": "Has the washing machine ever broken down?",\n    "options": [\n      "yes",\n      "no"\n    ]\n  },\n  {\n    "question": "What is the current physical condition of the washing machine?",\n    "options": [\n      "excellent",\n      "good",\n      "fair",\n      "poor"\n    ]\n  },\n  {\n    "question": "Does the washing machine have a warranty?",\n    "options": [\n      "yes",\n      "no"\n    ]\n  },\n  {\n    "question": "If the washing machine has a warranty, what is the warranty period?",\n    "options": [\n      "1 year",\n      "2 years",\n      "3 years",\n      "4 years",\n      "5 years"\n    ]\n  }\n]\n\n```'
DATA = parse_nested_json(SAMPLE)
# print(SAMPLE)
product = "washing machine"
ques = [e['question'] for e in DATA]
ans = [e['options'][0] for e in DATA]

bot_memory = QAMemory("question")
bot_memory.load_all(product, ques, ans)


AI21 ready
Enable tracing at assess
Load done


# Assess Electronic

In [3]:
from botcore.chains.assess_usage import build_assess_elec_usage
chain = build_assess_elec_usage(MODEL, bot_memory.memory)

# Pros and cons

In [2]:
from botcore.chains.pros_cons import build_pros_cons_chain
chain = build_pros_cons_chain(MODEL, bot_memory.memory)

# Router

In [2]:
from botcore.routing.chat_route import ProductChatRouter

router = ProductChatRouter(MODEL,bot_memory)
router.build_router()

Router ready
Build done


In [6]:
ans = router.chain({"input": "Please give a list of pros and cons."})
ans



> Entering new  chain...


/home/h4438/.local/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


pros and cons: {'input': 'Please give a list of pros and cons.', 'question': 'Please give a list of pros and cons.'}

> Entering new  chain...
Prompt after formatting:
You are a secondhand dealer and assessing the user's product. Based on your questions and user answers from the chat history.
 Human: I have this used washing machine of mine. Please ask me some questions about it.
AI: Has the washing machine ever broken down?
Human: yes
AI: What is the current physical condition of the washing machine?
Human: excellent
AI: Does the washing machine have a warranty?
Human: yes
AI: If the washing machine has a warranty, what is the warranty period?
Human: 1 year
 Please give your best answer.
 The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"pros": string  // a js array of the product's pros based on the chat history.
	"cons": string  // a js array of the product's cons based on the chat his

{'input': 'Please give a list of pros and cons.',
 'question': 'Please give a list of pros and cons.',
 'chat_history': 'Human: I have this used washing machine of mine. Please ask me some questions about it.\nAI: Has the washing machine ever broken down?\nHuman: yes\nAI: What is the current physical condition of the washing machine?\nHuman: excellent\nAI: Does the washing machine have a warranty?\nHuman: yes\nAI: If the washing machine has a warranty, what is the warranty period?\nHuman: 1 year',
 'text': '\n```\n{\n "pros": [\n  "excellent"\n ],\n "cons": [\n  "has broken down"\n ],\n "overview": "The washing machine is in good condition, but it has broken down before."\n}\n\n```'}